1. Install and Import Dependencies

In [163]:
!pip install torch torchvision torchaudio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [164]:
!pip install transformers requests beautifulsoup4 pandas numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [165]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

2. Instantiate Model

In [168]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment', clean_up_tokenization_spaces=True)

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

3. Encode and Calculate Sentiment

In [172]:
tokens = tokenizer.encode('This is amazing! I loved it! Great!', return_tensors='pt')

In [174]:
model(tokens)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.4722, -2.9249, -1.4977,  1.2982,  4.4999]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [177]:
result = model(tokens)

In [179]:
result.logits

tensor([[-2.4722, -2.9249, -1.4977,  1.2982,  4.4999]],
       grad_fn=<AddmmBackward0>)

In [181]:
int(torch.argmax(result.logits))+1

5

In [183]:
tokens2 = tokenizer.encode('This is terrible! I hated it!', return_tensors='pt')

In [185]:
result2 = model(tokens2)

In [187]:
int(torch.argmax(result2.logits))+1

1

In [209]:
tokens3 = tokenizer.encode('It was good but couldve been better.', return_tensors='pt')

In [211]:
result3 = model(tokens3)

In [213]:
result3.logits

tensor([[-1.9964,  0.1832,  2.2913,  1.2500, -1.4624]],
       grad_fn=<AddmmBackward0>)

In [215]:
int(torch.argmax(result3.logits))+1

3

4. Collect Reviews

In [222]:
# build a scraper
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [223]:
reviews

["Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!",
 "Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for not taking any photos. We all enjoyed our food, as well as our coffee and tea drinks.We were greeted immediately by a friendly server asking if we would like to sit inside or out. We said we would like inside, but weren't exactly sure how many were joining us yet- at least 4. 

5. Load Reviews into DataFrame and Score

In [227]:
import pandas as pd
import numpy as np

In [229]:
df = pd.DataFrame(np.array(reviews), columns =['review'])

In [231]:
df.head()

,review
0,Very cute coffee shop and restaurant. They hav...
1,Six of us met here for breakfast before our wa...
2,We came for brunch and they ran out of seven s...
3,Great place with delicious food and friendly s...
4,The food was delicious. The ricotta pancakes w...


In [233]:
df.tail()

,review
5,Great food amazing coffee and tea. Short walk ...
6,Ricotta hot cakes! These were so yummy. I ate ...
7,"Great service, lovely location, and really ama..."
8,We came for brunch twice in our week-long visi...
9,It was ok. Had coffee with my friends. I'm new...


In [249]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors = 'pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [251]:
sentiment_score(reviews[0])

5

In [253]:
sentiment_score(reviews[9])

3

In [257]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [261]:
df

,review,sentiment
0,Very cute coffee shop and restaurant. They hav...,4
1,Six of us met here for breakfast before our wa...,4
2,We came for brunch and they ran out of seven s...,2
3,Great place with delicious food and friendly s...,5
4,The food was delicious. The ricotta pancakes w...,4
5,Great food amazing coffee and tea. Short walk ...,5
6,Ricotta hot cakes! These were so yummy. I ate ...,5
7,"Great service, lovely location, and really ama...",5
8,We came for brunch twice in our week-long visi...,4
9,It was ok. Had coffee with my friends. I'm new...,3
